In [1]:
from IPython.display import clear_output
!pip install torch
!pip install torchvision
!pip install opencv-python
!pip install seaborn
!pip install matplotlib
!pip install cleverhans
!pip install cachetools
!pip install pandas
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y
clear_output()

In [2]:
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image

# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import gc
from load_diabetic import load_diabetic_data
    
# to make the results are reproducible
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# Dataset

In [4]:
train_loader, val_loader = load_diabetic_data()

# Model

In [5]:
num_classes = 5

model_ft = models.resnet50(pretrained=True)
model_ft.fc = nn.Linear(model_ft.fc.in_features, num_classes)
model = model_ft.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Train

In [6]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [7]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [8]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0);
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        if (i + 1) % 100 == 0:
            print('[train loss %.5f], [train acc %.5f]' % (train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    print('[Train loss %.5f], [Train acc %.5f]' % (total_loss_train[-1], total_acc_train[-1]))

In [9]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            gc.collect()
            torch.cuda.empty_cache()
            images, labels = data
            N = images.size(0)
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')

In [10]:
epochs = 2

for i in range(epochs):
    gc.collect()
    torch.cuda.empty_cache()
    print(i)
    train(train_loader, model, criterion, optimizer)
    print("Training Done")
    validate(val_loader, model, criterion, optimizer, i)

0
[train loss 0.93850], [train acc 0.65875]
[Train loss 0.93850], [Train acc 0.65875]
Training Done
------------------------------------------------------------
[epoch 0], [val loss 0.63502], [val acc 0.77135]
------------------------------------------------------------
1
[train loss 0.63918], [train acc 0.76000]
[Train loss 0.63918], [Train acc 0.76000]
Training Done
------------------------------------------------------------
[epoch 1], [val loss 0.55321], [val acc 0.79988]
------------------------------------------------------------


In [ ]:
epochs = 4

for i in range(epochs):
    gc.collect()
    torch.cuda.empty_cache()
    print(i)
    train(train_loader, model, criterion, optimizer)
    print("Training Done")
    validate(val_loader, model, criterion, optimizer, i)

0
[train loss 0.56550], [train acc 0.78500]
[Train loss 0.56550], [Train acc 0.78500]
Training Done
------------------------------------------------------------
[epoch 0], [val loss 0.48913], [val acc 0.81891]
------------------------------------------------------------
1
[train loss 0.51924], [train acc 0.81500]
[Train loss 0.51924], [Train acc 0.81500]
Training Done
------------------------------------------------------------
[epoch 1], [val loss 0.53319], [val acc 0.80668]
------------------------------------------------------------
2
[train loss 0.48673], [train acc 0.80750]
[Train loss 0.48673], [Train acc 0.80750]
Training Done
------------------------------------------------------------
[epoch 2], [val loss 0.50199], [val acc 0.81891]
------------------------------------------------------------
3
[train loss 0.44270], [train acc 0.84500]
[Train loss 0.44270], [Train acc 0.84500]
Training Done
------------------------------------------------------------
[epoch 3], [val loss 0.514

In [14]:
torch.save(model, 'model/model_diabetic.pth')